In [21]:
import pandas as pd
import json
from pandasql import sqldf
import os
from tabulate import tabulate

# Henter filsti
notebook_directory = os.getcwd()
root = os.path.abspath(os.path.join(notebook_directory, ".."))
file_path = os.path.join(root, "data", "Frost_Observations")

# Leser inn data fra fil
with open(file_path, "r") as file:
    data = json.load(file)

# Filtrerer relevant informasjon og konverterer til DataFrame
brukbar_data = [
    {
        "elementId": obs["elementId"],
        "sourceId": entry["sourceId"],
        "referenceTime": entry["referenceTime"],
        "value": obs["value"],
        "unit": obs["unit"]
    }
    for entry in data["data"]
    for obs in entry["observations"]
]

df = pd.DataFrame(brukbar_data)

# Håndtering av manglende verdier
df['value'] = pd.to_numeric(df['value'], errors='coerce')
df['value'] = df.groupby('elementId')['value'].transform(lambda x: x.fillna(x.mean()))

# Konverterer 'referenceTime' til datetime-format
df['referenceTime'] = pd.to_datetime(df['referenceTime'], errors='coerce')

# Gruppere og beregne gjennomsnittet for eventuelle duplikater
df_clean = df.groupby(["referenceTime", "elementId"], as_index=False)['value'].mean()

# Pivotere DataFrame for å få hver 'elementId' som en kolonne
df_pivot = df_clean.pivot(index="referenceTime", columns="elementId", values="value").reset_index()

# 1. Endre navn på kolonnene og konvertere til riktig format
df_pivot.rename(columns={
    "mean(air_temperature P1D)": "gjennomsnittstemperatur",
    "mean(relative_humidity P1D)": "gjennomsnittsfuktighet",
    "boolean_clear_sky_weather(cloud_area_fraction P1D)": "overskyet"
}, inplace=True)

# 2. Konvertere 'overskyet' til True/False
df_pivot["overskyet"] = df_pivot["overskyet"].apply(lambda x: True if x == 1 else False)

# 3. Beregn om temperaturen er over gjennomsnittet
avg_temp = df_pivot["gjennomsnittstemperatur"].mean()
df_pivot["er temperatur over gjennomsnitt?"] = df_pivot["gjennomsnittstemperatur"].apply(lambda x: True if x > avg_temp else False)

# 4. Fjern 'sky_condition' kolonnen
df_pivot.drop(columns=['sky_condition'], errors='ignore', inplace=True)

# 5. Bruke Pandas SQL (sqldf) for å hente data fra DataFrame
pysqldf = lambda q: sqldf(q, globals())

# Spørring 1: Gjennomsnittlig temperatur i juli per år
query = """
    SELECT strftime('%Y', referenceTime) AS Year,
           AVG(gjennomsnittstemperatur) AS avg_temperature
    FROM df_pivot
    WHERE strftime('%m', referenceTime) = '07'
    GROUP BY Year
"""
resultat_juli = pysqldf(query)

# Spørring 2: Alle dager med gjennomsnittstemperatur over 20 grader og klart vær
query2 = """
    SELECT * FROM df_pivot
    WHERE gjennomsnittstemperatur >= 20.0
    AND overskyet == False
    ORDER BY gjennomsnittstemperatur DESC;
"""
resultat_klart_vær = pysqldf(query2)

# 6. Vis resultater med tabulate for bedre utskrift

# Utskrift av pivotert DataFrame
print("Pivotert DataFrame med skytilstand og temperatur over gjennomsnitt:")
print(tabulate(df_pivot.head(), headers='keys', tablefmt='grid'))

# Utskrift av resultatene for juli-temperaturer
print("\nGjennomsnittlig temperatur i juli per år:")
print(tabulate(resultat_juli, headers='keys', tablefmt='grid'))

# Utskrift av resultatene for dager med temperatur over 20 grader og klart vær
print("\nDager med temperatur over 20 grader og klart vær:")
print(tabulate(resultat_klart_vær, headers='keys', tablefmt='grid'))


Pivotert DataFrame med skytilstand og temperatur over gjennomsnitt:
+----+---------------------------+-------------+---------------------------+--------------------------+------------------------------------+
|    | referenceTime             | overskyet   |   gjennomsnittstemperatur |   gjennomsnittsfuktighet | er temperatur over gjennomsnitt?   |
+====+===========================+=============+===========================+==========================+====================================+
|  0 | 2020-07-01 00:00:00+00:00 | False       |                     14.1  |                       66 | False                              |
+----+---------------------------+-------------+---------------------------+--------------------------+------------------------------------+
|  1 | 2020-07-02 00:00:00+00:00 | False       |                     13.75 |                       63 | False                              |
+----+---------------------------+-------------+---------------------------+----------